Step 1: Import and Read the File
python



In [6]:
import pandas as pd

# Read the Enron emails CSV
df = pd.read_csv("/Users/vaishnavipullakhandam/Desktop/github/Data_Analysis/Data QA Analysis/emails.csv",nrows=20000)

# Display the first few rows
print(df.head())

                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


Step 2: Extract Key Email Fields
- To extract Message-ID, Date, From, To, Subject, and Body.

In [7]:
# Function to extract parts from the raw email message
def extract_fields(message):
    lines = message.split("\n")  # Split the message into lines
    data = {
        "Message-ID": "",
        "Date": "",
        "From": "",
        "To": "",
        "Subject": "",
        "Body": ""
    }
    
    body_lines = []
    body_started = False
    
    for line in lines:
        line = line.strip()  # remove leading/trailing spaces
        if line.startswith("Message-ID:"):
            data["Message-ID"] = line[len("Message-ID:"):].strip()
        elif line.startswith("Date:"):
            data["Date"] = line[len("Date:"):].strip()
        elif line.startswith("From:"):
            data["From"] = line[len("From:"):].strip()
        elif line.startswith("To:"):
            data["To"] = line[len("To:"):].strip()
        elif line.startswith("Subject:"):
            data["Subject"] = line[len("Subject:"):].strip()
        elif line == "":
            body_started = True  # body starts after first blank line
        elif body_started:
            body_lines.append(line)
    
    data["Body"] = " ".join(body_lines)  # join body lines into one string
    return pd.Series(data)


Step 3: Apply the Function to Your DataFrame

In [8]:
# Apply the function to every row in the 'message' column
extracted_df = df["message"].apply(extract_fields)

# Combine extracted fields with the original DataFrame
df_cleaned = pd.concat([df, extracted_df], axis=1)

# Show the cleaned result
print(df_cleaned[['file', 'Message-ID', 'Date', 'From', 'To', 'Subject', 'Body']].head())


                       file                                     Message-ID  \
0     allen-p/_sent_mail/1.  <18782981.1075855378110.JavaMail.evans@thyme>   
1    allen-p/_sent_mail/10.  <15464986.1075855378456.JavaMail.evans@thyme>   
2   allen-p/_sent_mail/100.  <24216240.1075855687451.JavaMail.evans@thyme>   
3  allen-p/_sent_mail/1000.  <13505866.1075863688222.JavaMail.evans@thyme>   
4  allen-p/_sent_mail/1001.  <30922949.1075863688243.JavaMail.evans@thyme>   

                                    Date                     From  \
0  Mon, 14 May 2001 16:39:00 -0700 (PDT)  phillip.allen@enron.com   
1   Fri, 4 May 2001 13:51:00 -0700 (PDT)  phillip.allen@enron.com   
2  Wed, 18 Oct 2000 03:00:00 -0700 (PDT)  phillip.allen@enron.com   
3  Mon, 23 Oct 2000 06:13:00 -0700 (PDT)  phillip.allen@enron.com   
4  Thu, 31 Aug 2000 05:07:00 -0700 (PDT)  phillip.allen@enron.com   

                        To    Subject  \
0     tim.belden@enron.com              
1  john.lavorato@enron.com        

Step 4: Save the Cleaned Data

In [9]:
# Save the cleaned data to a new CSV
df_cleaned.to_csv("emails_cleaned.csv", index=False)


Step 5: Clean the Email Body 

In [10]:
# Clean the body: remove multiple spaces, lowercase, and trim
df_cleaned["Body_cleaned"] = df_cleaned["Body"].str.lower().str.strip().str.replace(r"\s+", " ", regex=True)

# Show sample cleaned body
print(df_cleaned[["Body", "Body_cleaned"]].head(2))


                                                Body  \
0                               Here is our forecast   
1  Traveling to have a business meeting takes the...   

                                        Body_cleaned  
0                               here is our forecast  
1  traveling to have a business meeting takes the...  


Step 6: Detect and Report Missing Fields (QA Check)

In [11]:
# Check for missing values in key fields
missing_summary = df_cleaned[["Message-ID", "Date", "From", "To", "Subject", "Body"]].isnull().sum()

print("🔍 Missing field count:")
print(missing_summary)


🔍 Missing field count:
Message-ID    0
Date          0
From          0
To            0
Subject       0
Body          0
dtype: int64


In [12]:
# Flag rows with missing essential fields
df_cleaned["has_missing_fields"] = df_cleaned[["From", "To", "Subject", "Body"]].isnull().any(axis=1)

# Display some rows with missing info
print(df_cleaned[df_cleaned["has_missing_fields"]].head())


Empty DataFrame
Columns: [file, message, Message-ID, Date, From, To, Subject, Body, Body_cleaned, has_missing_fields]
Index: []


Step 7: Categorize Email Types (Manual Labeling)


In [14]:
def label_category(body):
    body = body.lower()
    if "meeting" in body or "schedule" in body:
        return "Meeting"
    elif "thank you" in body or "thanks" in body:
        return "Gratitude"
    elif "invoice" in body or "payment" in body:
        return "Finance"
    elif "complaint" in body or "issue" in body:
        return "Complaint"
    elif "test" in body:
        return "Test"
    else:
        return "Other"


In [15]:
df_cleaned["Category"] = df_cleaned["Body_cleaned"].apply(label_category)

# Show a sample
print(df_cleaned[["Subject", "Body_cleaned", "Category"]].head(10))


                                             Subject  \
0                                                      
1                                                Re:   
2                                           Re: test   
3                                                      
4                                          Re: Hello   
5                                          Re: Hello   
6                                                      
7                       Re: PRC review - phone calls   
8                     Re: High Speed Internet Access   
9  FW: fixed forward or other Collar floor gas pr...   

                                        Body_cleaned   Category  
0                               here is our forecast      Other  
1  traveling to have a business meeting takes the...    Meeting  
2                      test successful. way to go!!!       Test  
3  randy, can you send me a schedule of the salar...    Meeting  
4                  let's shoot for tuesday at 11:45. 

 Step 8: Monitor Sender Trends (Simple Analytics)

In [16]:
email_counts = df_cleaned["From"].value_counts().reset_index()
email_counts.columns = ["Sender", "Email_Count"]

print(email_counts.head())


                          Sender  Email_Count
0            eric.bass@enron.com         3219
1          john.arnold@enron.com         2371
2        phillip.allen@enron.com         1809
3  enron.announcements@enron.com          263
4         bryant@cheatsheets.net          205


In [17]:
# Any sender who sent more than 1000 emails?
print(email_counts[email_counts["Email_Count"] > 1000])


                    Sender  Email_Count
0      eric.bass@enron.com         3219
1    john.arnold@enron.com         2371
2  phillip.allen@enron.com         1809
